In [177]:
import zipfile
zip_path = 'Data_Cleaned-20230727T065311Z-001.zip'
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall('RediMinds/')

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

In [2]:
gpu_id = 0  # Change this to the index of the desired GPU
torch.cuda.set_device(gpu_id)

In [3]:
DATADIR = "RediMinds/Data_Cleaned/"
categories = ["Anaesthesia_machine","baby_incubator","Bone_saws","C_arm","colonoscope","Curved_Mayo_Scissor","difibrillator","Electrocautery_devices","fetal_doppler","forceps","Heart_Lung_Machine","IABP","IMRT","infusion_pump","Laryngoscopes","mayfield_clamp","Needle_Biopsy_Device","phacoemulsification","Radiofrequency_Ablation_Device","Scalpel","Straight_Dissection_Clamp","Straight_Mayo_Scissor","Suction_Machine","ventilator","x_ray"]
input_size = (224, 224)

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, categories, input_size, transform=None):
        self.data_dir = data_dir
        self.categories = categories
        self.input_size = input_size
        self.transform = transform
        self.data = []
        self.labels = []
        self.load_data()

    def load_data(self):
        for cat in self.categories:
            path = os.path.join(self.data_dir, cat)
            class_num = self.categories.index(cat)
            for img_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, img_name)
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(self.input_size)
                    if self.transform:
                        img = self.transform(img)
                    self.data.append(img)
                    self.labels.append(class_num)
                except Exception as e:
                    pass

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        return img, label

In [5]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    #transforms.CenterCrop(10),
    # transforms.RandomRotation(15),
    # transforms.ColorJitter(brightness=0.2, contrast = 0.25, saturation = 0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

dataset = CustomDataset(DATADIR, categories, input_size, transform=transform)

train_size = int(0.70 * len(dataset))
validation_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - validation_size
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,validation_size, test_size])

In [6]:
print(train_size,validation_size,test_size)

3344 716 718


In [7]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomGooglenet(nn.Module):
    def __init__(self, num_classes):
        super(CustomGooglenet, self).__init__()
        googlenet = models.googlenet(pretrained=models.GoogLeNet_Weights.IMAGENET1K_V1)
        #googlenet = models.googlenet(pretrained='googlenet')


        # Freeze all layers by default
        for param in googlenet.parameters():
            param.requires_grad = False

        # for param in googlenet.inception5a.parameters():
        #     param.requires_grad = True

        for param in googlenet.inception5b.parameters():
            param.requires_grad = True

        self.googlenet = nn.Sequential(*list(googlenet.children())[:-1])

        self.cnnl = nn.Sequential(
            nn.Conv2d(1024,512, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(512,512,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Conv2d(512,128,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.AdaptiveAvgPool2d((1, 1))
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024,512, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(512,512,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.Conv2d(512,128,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.AdaptiveAvgPool2d((1, 1))
        )
        #self.cnnl = nn.Conv2d(512, num_classes, kernel_size=3,padding=1)
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes),
            # nn.ReLU(),
            # nn.Dropout(0.6),
            # nn.Linear(2048,1024),
            # nn.ReLU(),
            #nn.Linear(1024,1024),
            #nn.ReLU(),
            #nn.Dropout(0.3)
            # nn.Linear(4096, num_classes)
            )

    def forward(self, x):
        x = self.googlenet(x)
        x = x.view(x.size(0), -1,1,1)
        x = self.cnnl(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        # x = self.cnnl(x)
        # x = torch.flatten(x,1)
        return x

In [39]:
class CustomGooglenet(nn.Module):
    def __init__(self, num_classes):
        super(CustomGooglenet, self).__init__()
        googlenet = models.googlenet(pretrained=models.GoogLeNet_Weights.IMAGENET1K_V1)
        #googlenet = models.googlenet(pretrained='googlenet')


        # Freeze all layers by default
        for param in googlenet.parameters():
            param.requires_grad = False

        # for param in googlenet.inception5a.parameters():
        #     param.requires_grad = True

        for param in googlenet.inception5b.parameters():
            param.requires_grad = True

        self.googlenet = nn.Sequential(*list(googlenet.children())[:-1])

        self.cnnl = nn.Sequential(
            nn.Conv2d(1024,4098, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(4098),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Conv2d(4098,4098,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(4098),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Conv2d(4098,2048,kernel_size = 3, padding = 1),
            nn.BatchNorm2d(2048),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.AdaptiveAvgPool2d((1, 1))
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024, 1024, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(1024,512, kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Conv2d(512,512,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.Conv2d(512,128,kernel_size = 3, padding = 1),
            # nn.ReLU(),
            # nn.Dropout(0.4),
            # nn.AdaptiveAvgPool2d((1, 1))
        )
        #self.cnnl = nn.Conv2d(512, num_classes, kernel_size=3,padding=1)
        self.head = nn.Sequential(
            nn.Linear(2048, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.6),
            # nn.Linear(4098, 2048),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Linear(2048, 2048),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(2048,num_classes),
            # nn.ReLU(),
            # nn.Dropout(0.6),
            # nn.Linear(2048,1024),
            # nn.ReLU(),
            #nn.Linear(1024,1024),
            #nn.ReLU(),
            #nn.Dropout(0.3)
            # nn.Linear(4096, num_classes)
            )

    def forward(self, x):
        x = self.googlenet(x)
        x = x.view(x.size(0), -1,1,1)
        x = self.cnnl(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        # x = self.cnnl(x)
        # x = torch.flatten(x,1)
        return x

weight_enum = torch.hub.load("pytorch/vision", "get_model_weights", name="googlenet")
print([weight for weight in weight_enum])

In [40]:
from torch.optim.lr_scheduler import StepLR

num_classes = len(categories)
model = CustomGooglenet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size = 20, gamma=0.5)

In [41]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

googlenet.0.conv.weight: requires_grad=False
googlenet.0.bn.weight: requires_grad=False
googlenet.0.bn.bias: requires_grad=False
googlenet.2.conv.weight: requires_grad=False
googlenet.2.bn.weight: requires_grad=False
googlenet.2.bn.bias: requires_grad=False
googlenet.3.conv.weight: requires_grad=False
googlenet.3.bn.weight: requires_grad=False
googlenet.3.bn.bias: requires_grad=False
googlenet.5.branch1.conv.weight: requires_grad=False
googlenet.5.branch1.bn.weight: requires_grad=False
googlenet.5.branch1.bn.bias: requires_grad=False
googlenet.5.branch2.0.conv.weight: requires_grad=False
googlenet.5.branch2.0.bn.weight: requires_grad=False
googlenet.5.branch2.0.bn.bias: requires_grad=False
googlenet.5.branch2.1.conv.weight: requires_grad=False
googlenet.5.branch2.1.bn.weight: requires_grad=False
googlenet.5.branch2.1.bn.bias: requires_grad=False
googlenet.5.branch3.0.conv.weight: requires_grad=False
googlenet.5.branch3.0.bn.weight: requires_grad=False
googlenet.5.branch3.0.bn.bias: req

In [ ]:
train_acc_history = []
train_loss_history = []
val_acc_history = []
val_loss_history = []

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss, train_corrects = 0.0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        labels = torch.squeeze(labels)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels).item()

    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects / len(train_dataset)
    train_acc_history.append(train_acc)
    train_loss_history.append(train_loss)

    model.eval()
    val_loss, val_corrects = 0.0, 0.0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels).item()

    val_loss = val_loss / len(validation_dataset)
    val_acc = val_corrects / len(validation_dataset)

    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)

    # scheduler.step()
    
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

Epoch 1/50 | Train Loss: 2.8958 | Train Acc: 0.3155 | Val Loss: 3.6114 | Val Acc: 0.5126
Epoch 2/50 | Train Loss: 2.0912 | Train Acc: 0.5317 | Val Loss: 3.1635 | Val Acc: 0.6089
Epoch 3/50 | Train Loss: 2.1118 | Train Acc: 0.6017 | Val Loss: 1.8981 | Val Acc: 0.6313
Epoch 4/50 | Train Loss: 1.8166 | Train Acc: 0.6941 | Val Loss: 0.9847 | Val Acc: 0.7416
Epoch 5/50 | Train Loss: 1.4718 | Train Acc: 0.7315 | Val Loss: 0.8797 | Val Acc: 0.7514
Epoch 6/50 | Train Loss: 1.5934 | Train Acc: 0.7548 | Val Loss: 1.2831 | Val Acc: 0.7109
Epoch 7/50 | Train Loss: 1.5099 | Train Acc: 0.7808 | Val Loss: 1.2590 | Val Acc: 0.7514
Epoch 8/50 | Train Loss: 1.4174 | Train Acc: 0.7892 | Val Loss: 1.0361 | Val Acc: 0.7723
Epoch 9/50 | Train Loss: 1.2833 | Train Acc: 0.8035 | Val Loss: 0.9758 | Val Acc: 0.7332
Epoch 10/50 | Train Loss: 1.0362 | Train Acc: 0.8379 | Val Loss: 1.1197 | Val Acc: 0.7612
Epoch 11/50 | Train Loss: 0.9239 | Train Acc: 0.8535 | Val Loss: 1.6213 | Val Acc: 0.7402
Epoch 12/50 | Train

In [ ]:
# Plot training and validation accuracy
import matplotlib.pyplot as plt

plt.plot(range(1, num_epochs+1), train_acc_history, label='Train')
plt.plot(range(1, num_epochs+1), val_acc_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(range(1, num_epochs+1), train_loss_history, label='Train')
plt.plot(range(1, num_epochs+1), val_loss_history, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()
test_loss, test_corrects = 0.0, 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels).item()

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects / len(test_dataset)

print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

---

In [193]:
torch.save(model.state_dict(),"model_torch_googlenet_2.pth")

In [194]:
len(categories)

25